# Final Project - Heat Diffusion

## Tyler Percy

### What I was investigating...

The purpose of this project was to experiment with heat diffusion across a 2D, and eventually, 3D plane.  Specifically, I was curious in seeing how different dimensional planes, initial conditions, and boundary conditions, affected metrics such as the rate of heat transfer from cell to cell.

### Initial Conditions...

The initial conditions I experimented with were a 2 dimensional plane, where the heat source was constant from the top left corner of a 12x12 grid.

The initial location of the heat source can be changed via the INIT_LOC constant in the 2D.py code, or in the jupyter implementation you will see below

### The Code!

Disclaimer: After walking through the code once in this notebook, I would recommend running it using the .py files that are also included in this repo, as they generally produce higher quality outputs.

In [2]:
from vpython import *

<IPython.core.display.Javascript object>

### Constants 

These variables can be changed to experiment with different initial conditions.

for example, n is how many iterations you want the program to run for,
x and y are the size of your grid, but they must stay as the same number

In [3]:
DEBUG_MODE = 0
CONSTANT_HEAT_SOURCE = 1 # Keep this as 1 for now
RATE = 4
INIT_LOC = 0 # can be anything from 0 to x
n = 120
r = 0.1  
x = 12   
y = 12

### Print Grid

This function outputs the new iteration of the grid to the screen, with the correctly updated heat values

In [4]:
def print_grid(grid, x, y):
    for i in range(0, x):
        for j in range(0, y):
            print("{:.02f}\t".format(grid[i][j]), end = "")
        print()

### Get New Cell Temp

The bulk of the calculations are done here.  The if statements handle all of the potential conditions in which temperature needs to be updated.  The reason there are so many of them is because depending on the amount of adjacent squares, you will need to use a formula with slightly tweaked values to ensure the calculations are accurate.



In [5]:
def get_new_cell_temp(temps, r, i, j, x, y):

    if DEBUG_MODE:
        print(f"x: {i} y: {j}")

    # starting coordinate
    if i == j == 0:
        delta_ti = (2 * r * (((temps[i+1][j] + temps[i][j+1]) / 2) - temps[i][j]))
    # along 0 < j < y-1
    elif j == 0 and 0 < i < x-1:
        delta_ti = (3 * r * (((temps[i-1][j] + temps[i+1][j] + temps[i][j+1]) / 3) - temps[i][j]))
    # along 0 < i < x-1
    elif i == 0 and 0 < j < y-1:
        delta_ti = (3 * r * (((temps[i+1][j] + temps[i][j-1] + temps[i][j+1]) / 3) - temps[i][j]))
    # final coordinate
    elif i == x-1 and j == y-1:
        delta_ti = (2 * r * (((temps[i-1][j] + temps[i][j-1]) / 2) - temps[i][j]))
    # along j = y-1
    elif j == y-1:
        delta_ti = (3 * r * (((temps[i+1][j] + temps[i-1][j] + temps[i][j-1]) / 3) - temps[i][j]))
    # along i = x-1
    elif i == x-1:
        delta_ti = (3 * r * (((temps[i-1][j] + temps[i][j+1] + temps[i][j-1]) / 3) - temps[i][j]))
    # any other coordinate touched by 4 adjacent cells
    else:
        delta_ti = (4 * r * (((temps[i-1][j] + temps[i+1][j] + temps[i][j-1] + temps[i][j+1]) / 4) - temps[i][j]))

    return delta_ti

### Is Edge

This function simply calculates if the cell in question is on the boundary of the grid.

In [6]:
def is_edge(grid, i, j, x, y):
    if i == 0 or j == 0 or i == x-1 or j == y-1:
        return True
    else:
        return False

### Main

Handles the setup and execution of the logical functions above.

##### Running this next cell after running the above cells will produce an output

In [ ]:
def main():

    ''' Initial conditions for the grids '''

    scene = canvas(center= vec(5,-5,0), userzoom= False)
    lbl = label(pos= vec(1,2,0), text= "Processing...")
    lbl_temps = label(pos= vec(8,2,0))
    boxes = [[box(pos= vec(i, -j, 0)) for i in range(x)] for j in range(y)]

    for i in range(x):
        for j in range(y):
            boxes[i][j].rotate(angle= 180,axis= vec(0,1,0))

    grid = [[0 for _ in range(x)] for _ in range(y)]
    temps = [[0 for _ in range(x)] for _ in range(y)]

    grid[INIT_LOC][INIT_LOC] = 100
    temps[INIT_LOC][INIT_LOC] = 100
    boxes[INIT_LOC][INIT_LOC].color = vec(50,0,0)

    # default grid
    if DEBUG_MODE:
        print_grid(grid, x, y)

    for i in range(n):
        rate(RATE)
        # Print current iteration of main loop
        if DEBUG_MODE:
            print(f"Iteration: {i+1}")

        for i in range(x):
            for j in range(y):
                if CONSTANT_HEAT_SOURCE:
                    grid[INIT_LOC][INIT_LOC] = 100
                    temps[INIT_LOC][INIT_LOC] = 100
                    boxes[INIT_LOC][INIT_LOC].color = vec(50,0,0)
                delta_ti = get_new_cell_temp(temps, r, i, j, x, y)

                # print return value of update_cell_temp
                if DEBUG_MODE:
                    print(f"delta_ti: {delta_ti}")

                grid[i][j] += delta_ti
                boxes[i][j].color += vec(delta_ti/10,delta_ti/25,delta_ti/50)

        temps = grid

    lbl.text = "Finished."

    if DEBUG_MODE:
        print()
        print_grid(grid, x, y)

    while(True):
        for i in range(x):
            for j in range(y):
                if boxes[i][j] == scene.mouse.pick:
                    lbl_temps.text = "{:.02f}% heated".format(temps[i][j])


if __name__ == "__main__":
    main()


### Output

As you can see, the heat spread from the top corner of the grid, until all the iterations were complete.  You may notice how it did not fill up the entire grid.  This is because of two potential factors.

A.  As the heat begins to spread out across the grid, more calculations must be made in order to transfer heat to adjacent cells.  Since more and more cells are being contacted by other cells, it means less heat is being transfered to each cell, meaning the process takes much longer the more cells are being operated on.

B.  It is also possible that there are not enough iterations in the program in order to sufficiently heat those bottom right cells.  This is easily amendable by increasing the amount of iterations using the n variable under constants.

### Lets try a different condition!

In [1]:
def main():

    ''' Initial conditions for the grids '''

    scene = canvas(center= vec(5,-5,0), userzoom= False)
    lbl = label(pos= vec(1,2,0), text= "Processing...")
    lbl_temps = label(pos= vec(8,2,0))
    boxes = [[box(pos= vec(i, -j, 0)) for i in range(x)] for j in range(y)]

    for i in range(x):
        for j in range(y):
            boxes[i][j].rotate(angle= 180, axis= vec(0,1,0))

    grid = [[0 for _ in range(x)] for _ in range(y)]
    temps = [[0 for _ in range(x)] for _ in range(y)]

    for i in range(x):
        for j in range(y):
            if is_edge(grid, i, j, x, y):
                grid[i][j] = 100
                temps[i][j] = 100
                boxes[i][j].color = vec(50,0,0)

    # default grid
    if DEBUG_MODE:
        print_grid(grid, x, y)

    for i in range(n):
        rate(RATE)
        # Print current iteration of main loop
        if DEBUG_MODE:
            print(f"Iteration: {i+1}")

        for i in range(x):
            for j in range(y):
                if is_edge(grid, i, j, x, y):
                    grid[i][j] = 100
                    temps[i][j] = 100
                    boxes[i][j].color = vec(50,0,0)
                delta_ti = get_new_cell_temp(temps, r, i, j, x, y)

                # print return value of update_cell_temp
                if DEBUG_MODE:
                    print(f"delta_ti: {delta_ti}")

                grid[i][j] += delta_ti
                boxes[i][j].color += vec(delta_ti/50,delta_ti/100,delta_ti/200)

        temps = grid

    lbl.text = "Finished."

    if DEBUG_MODE:
        print()
        print_grid(grid, x, y)

    while(True):
        for i in range(x):
            for j in range(y):
                if boxes[i][j] == scene.mouse.pick:
                    lbl_temps.text = "{:.02f}% heated".format(temps[i][j])


if __name__ == "__main__":
    main()

NameError: name 'canvas' is not defined

### Output

As you can see, the initial conditions for this simulation were very different.  Instead of having one constant heat source coming from the top left corner, the entire boundary of the grid was a constant heat source.  This caused the entire square to quickly become nearly fully heated.

### Okay, one more...

In [ ]:
def main():

    ''' Initial conditions for the grids '''

    scene = canvas(center= vec(5,-5,0), userzoom= False)
    lbl = label(pos= vec(1,2,0), text= "Processing...")
    lbl_temps = label(pos= vec(8,2,0))
    boxes = [[box(pos= vec(i, -j, 0)) for i in range(x)] for j in range(y)]

    for i in range(x):
        for j in range(y):
            boxes[i][j].rotate(angle= 180,axis= vec(0,1,0))

    grid = [[0 for _ in range(x)] for _ in range(y)]
    temps = [[0 for _ in range(x)] for _ in range(y)]

    grid[6][6] = 100
    temps[6][6] = 100
    boxes[6][6].color = vec(50,0,0)

    # default grid
    if DEBUG_MODE:
        print_grid(grid, x, y)

    for i in range(n):
        rate(RATE)
        # Print current iteration of main loop
        if DEBUG_MODE:
            print(f"Iteration: {i+1}")

        for i in range(x):
            for j in range(y):
                if CONSTANT_HEAT_SOURCE:
                    grid[6][6] = 100
                    temps[6][6] = 100
                    boxes[6][6].color = vec(50,0,0)
                delta_ti = get_new_cell_temp(temps, r, i, j, x, y)

                # print return value of update_cell_temp
                if DEBUG_MODE:
                    print(f"delta_ti: {delta_ti}")

                grid[i][j] += delta_ti
                boxes[i][j].color += vec(delta_ti/10,delta_ti/25,delta_ti/50)

        temps = grid

    lbl.text = "Finished."

    if DEBUG_MODE:
        print()
        print_grid(grid, x, y)

    while(True):
        for i in range(x):
            for j in range(y):
                if boxes[i][j] == scene.mouse.pick:
                    lbl_temps.text = "{:.02f}% heated".format(temps[i][j])


if __name__ == "__main__":
    main()

### Output

So here, we decided to start towards the middle of the grid, and watch the heat expand outwards.  What I found very interesting was how quickly the heat cascaded out from the center as opposed from the corner.  

I deduced that because the heat was able to spread to 4 adjacent squares initially, instead of 2, the rate at which it was able to heat up the grid was twice as fast, since it could hit new squares twice as quickly at a constant 100 heat rate.

### Conclusions!

It appears the rate of heat transfer directly correlated to a few key factors:

A.  The amount of iterations the simulation would run.

B.  The initial starting location of the constant heat source.

C.  The amount of constant heat sources present on the grid.

### How can we improve this experiment in the future?

A few things I would have liked to finish if I had more time:

A.  A fully working 3D representation

B.  A boundary condition where the heat would 'wrap' around edges and continue over corners.

C.  Including diagonals into the equation so adjacency was considered to be up to 8, instead of 4 cells.

